In [ ]:
import os
import cv2
import numpy as np
import torch

#import torchvision.transforms as transforms
from torch import nn
from torchvision import datasets
import transforms as T
#from torchvision import transforms

#from torchvision.models import segmentation as models

from sklearn.metrics import jaccard_similarity_score as jsc

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from task_3_utils import evaluate

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('Found GPU ', device)

In [ ]:
sig = nn.Sigmoid()

In [ ]:
#from torchvision.models.segmentation import fcn_resnet50
#net = fcn_resnet50(num_classes=1).cuda()
#net = models.__dict__['deeplabv3_resnet50'](num_classes=1, pretrained=True)

In [ ]:
#resume = '/scratch/gallowaa/cciw/logs/v0.1.0/checkpoint/fcn_resnet50_bs40_wd5e-04_aug1_1.ckpt'

#root = '/scratch/gallowaa/cciw/logs/v0.2.4/checkpoint'
#ckpt_file = 'fcn_resnet50_bs40_wd5e-04_def_1.ckpt'

root = '/scratch/gallowaa/cciw/logs/Lab/v1-36-img/checkpoint'
ckpt_file = 'unet_bn_bs32_wd5e-04_pytorch_unet_v4_bi_1.ckpt'

print('==> Resuming from checkpoint..')
checkpoint = torch.load(os.path.join(root, ckpt_file))
net = checkpoint['net']
best_acc = checkpoint['loss']
start_epoch = checkpoint['epoch'] + 1
torch.set_rng_state(checkpoint['rng_state'])

In [ ]:
#net = models.__dict__['fcn_resnet50'](num_classes=1)
#net = net.cuda()

In [ ]:
from unet import UNet
#bilinear = True if args.bilinear else False
net = UNet(n_channels=3, n_classes=1, bilinear=True).to(device)

In [ ]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))
print(model_names)

In [ ]:
#test_file = 'GLNI_1356-3_2017-08-15_image-1_crop.jpg'
#test_file = 'GLNI_1352-1_2017-08-16_image-1_crop.jpg'
#test_file = 'GLNI_1346-2_2017-08-17_image-1_crop.jpg'
#root_path = '/scratch/ssd/gallowaa/cciw/dataset_raw_v0-2-x/Train/2017-08/'
root_path = '/scratch/ssd/gallowaa/cciw/dataset_raw/Test/Lab/done/'

#img = cv2.imread(os.path.join(root_path, test_file))
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss() # sigmoid cross entropy

mytforms = []
mytforms.append(T.RandomCrop(224))
mytforms.append(T.RandomHorizontalFlip(0.5))
mytforms.append(T.RandomVerticalFlip(0.5))
mytforms.append(T.ToTensor())
mytforms.append(T.Normalize((0.2613, 0.2528, 0.2255), 
                            (0.2637, 0.2546, 0.2306))) 
mytforms = T.Compose(mytforms)

test_tform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.2613, 0.2528, 0.2255), # mean (RGB)
                (0.2637, 0.2546, 0.2306))
])

In [ ]:
# for debugging purposes
trainset = datasets.VOCSegmentation(
    root='/scratch/ssd/gallowaa/cciw/Lab', year='2012', 
    image_set='train', download=False,
    transforms=mytforms)

In [ ]:
# for debugging purposes
valset = datasets.VOCSegmentation(
    root='/scratch/ssd/gallowaa/cciw/Lab', year='2012', 
    image_set='val', download=False,
    transforms=test_tform)
    
bs = 75
valloader = torch.utils.data.DataLoader(valset, batch_size=bs, shuffle=False)

In [ ]:
from task_3_utils import evaluate

In [ ]:
val_iou, val_loss = evaluate(net, valloader, loss_fn, device)

In [ ]:
y_true = np.array([[0, 0, 0],
                   [1, 1, 0]])

y_pred = np.array([[1, 1, 1],
                   [1, 0, 0]])

In [ ]:
y_true.shape

In [ ]:
jsc(y_true[0], y_pred[0])

In [ ]:
pred_np.round()[:, 0].reshape(pred_np.shape[0], -1).shape

In [ ]:
targets_np.reshape(targets_np.shape[0], -1).shape

In [ ]:
#sig

In [ ]:
running_iou = 0
running_loss = 0
with torch.no_grad():
    for i, (inputs, targets) in enumerate(valloader):
        targets_np = targets.numpy()
        inputs, targets = inputs.to(device), targets.to(device)
        pred = sig(net(inputs))

        # dataloader outputs targets with shape NHW, but we need NCHW
        batch_loss = loss_fn(pred, targets.unsqueeze(dim=1).float())
        running_loss += batch_loss.item()

        # jaccard similarity (IoU) on CPU
        pred_np = pred.detach().cpu().numpy()
        print(i, pred_np.shape)
        print(i, targets_np.shape)

        # flatten predictions and targets for IoU calculation
        running_iou += jsc(
            pred_np.round()[:, 0].reshape(pred_np.shape[0], -1),
        targets_np.reshape(targets_np.shape[0], -1))

In [ ]:
targets.squeeze().shape

In [ ]:
#np.argmax(pred_np, axis=1)

In [ ]:
#ct = 0
mean_iou = 0
with torch.no_grad():    
    for batch, (inputs, targets) in enumerate(valloader):
        inputs = inputs.cuda()
        targets_np = targets.detach().cpu().numpy()
        pred = sig(net(inputs))
        pred_np = pred.detach().cpu().numpy()       
        lbl = targets_np.reshape(targets_np.shape[0], -1)
        out = pred_np.round()[:, 0].reshape(pred_np.shape[0], -1)
        iou = jsc(out, lbl)
        mean_iou += iou
        #print(batch, iou)
mean_iou /= len(valloader)
print('%.4f' % mean_iou)

In [ ]:
import time

In [ ]:
start_time = time.time()

In [ ]:
time.time()-start_time

In [ ]:
import glob
#files = os.listdir(root_path)
files = glob.glob(root_path + '*.jpg')

w = 224
sy = 275
sx = 1050
plt.imshow(img[sy:sy + w, sx:sx + w, :])

#for i in range(3):
i = 2
img = cv2.imread(os.path.join(root_path, files[i]))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
imgc = img[sy:sy + w, sx:sx + w, :]
imgt = torch.FloatTensor(imgc).to(device)
imgt = imgt / imgt.max()
imgt = imgt.unsqueeze(0)
inputs_nchw = imgt.permute(0, 3, 1, 2)

with torch.no_grad():
    pred = sig(net(inputs_nchw))
inputs_nhwc = inputs_nchw.permute(0, 2, 3, 1)

idx = 0
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
ax1.imshow(inputs_nhwc.detach().cpu().numpy()[idx])
ax2.imshow(pred.detach().cpu().numpy()[idx, 0])
ax3.imshow(inputs_nhwc.detach().cpu().numpy()[idx], alpha=0.75)
ax3.imshow(pred.detach().cpu().numpy()[idx, 0], alpha=0.5)
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
plt.tight_layout()
#fig.savefig('img/' + ckpt_file.split('.')[0] + '_' + files[i].split('.')[0] + '_full.png')

In [ ]:
len(valloader) * 20

In [ ]:
with torch.no_grad():
    pred = sig(net(inputs))

inputs_nhwc = inputs.permute(0, 2, 3, 1)
inputs_nhwc_np = inputs_nhwc.detach().cpu().numpy()    
pred_np = pred.detach().cpu().numpy()
targets_np = targets.detach().cpu().numpy()

In [ ]:
def eval_binary_iou(outputs, targets, eps=1e-6):
    """Returns the average binary intersection-over-union score.
    Similar to sklearn.metrics.jaccard_similarity_score.
    @param outputs are model predictions (post-sigmoid) in Nx1xHxW format.
    @param targets are the labels in NxHxW format.
    @param eps is a small constant to prevent division by zero.
    """
    outputs = outputs.squeeze(1).round().long()
    # zero if output=0 or pred=0
    intersection = (outputs & targets).float().sum((1, 2))
    union = (outputs | targets).float().sum((1, 2))
    iou = intersection / (union + eps)
    return iou

In [ ]:
eval_binary_iou(pred, targets)

In [ ]:
lbl = targets_np.reshape(bs, -1)
out = pred_np.round()[:, 0].reshape(bs, -1)
jscv = jsc(out, lbl)
print(jscv)

In [ ]:
import pydensecrf.densecrf as dcrf
import pydensecrf.utils as utils

In [ ]:
utils.unary_from_softmax?

In [ ]:
def run_crf(inputs_nhwc_np, pred_np, idx):
    MAX_ITER = 10
    labels = np.stack([pred_np[idx, 0], 1 - pred_np[idx, 0]])
    c, h, w = labels.shape[0], labels.shape[1], labels.shape[2]
    labels = labels.astype('float') / labels.max()
    U = utils.unary_from_softmax(labels)
    U = np.ascontiguousarray(U)
    d = dcrf.DenseCRF2D(w, h, c)
    d.setUnaryEnergy(U)
    """
    @param compat=3, Potts model - it introduces a penalty for nearby similar 
    pixels that are assigned different labels. 
    """
    # This adds the color-independent term, features are the locations only.
    d.addPairwiseGaussian(sxy=3, compat=3)
    # This adds the color-dependent term, i.e. features are (x,y,r,g,b).
    # im is an image-array, e.g. im.dtype == np.uint8

    mean = (0.2613, 0.2528, 0.2255), # mean (RGB)
    std  = (0.2637, 0.2546, 0.2306)
    image = (((inputs_nhwc_np[idx] * std) + mean) * 255).astype('uint8')

    d.addPairwiseBilateral(sxy=80, srgb=13, rgbim=image, compat=10)
    Q = d.inference(MAX_ITER)
    Q = np.array(Q).reshape((c, h, w))
    # binarize output
    Q[0][Q[0] >= 0.5] = 1
    Q[0][Q[0] < 0.5] = 0
    
    return Q[0, :, :]

In [ ]:
mean = (0.2613, 0.2528, 0.2255), # mean (RGB)
std  = (0.2637, 0.2546, 0.2306)
image = (((inputs_nhwc_np[idx] * std) + mean) * 255).astype('uint8')
print(image.min())
print(image.max())

In [ ]:
pred_np.max()

In [ ]:
plt.imshow(Q[0, :, :])

In [ ]:
figsize=(12, 2.5)
#figsize=(6, 3)
#idx = 2 # train mode
for idx in range(len(pred_np)):
    if idx == 0 or idx == 5:
        
        pred_crf = run_crf(inputs_nhwc_np, pred_np, idx)
        
        fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=figsize)
        ax1.imshow(inputs_nhwc_np[idx])
        ax2.imshow(pred_np[idx, 0])
        ax3.imshow(pred_np.round()[idx, 0])
        ax4.imshow(pred_crf)
        ax5.imshow(targets_np[idx])
        lbl = targets_np[idx].reshape(1, -1)
        out = pred_np.round()[idx, 0].reshape(1, -1)
        out_crf = pred_crf.reshape(1, -1)
        ax1.set_title('Image'); ax2.set_title('Logits') 
        ax3.set_title('IoU %.2f' % jsc(out, lbl)); 
        ax4.set_title('IoU %.2f' % jsc(out_crf, lbl))
        ax5.set_title('Label')
        pretty_image([ax1, ax2, ax3, ax4, ax5])
        plt.savefig(ckpt_file.split('/')[-1].split('.')[0] + '_lab_val_demo' + '_%d.png' % idx)

In [ ]:
len(trainloader)

In [ ]:
bs = 1
trainloader = torch.utils.data.DataLoader(trainset, batch_size=bs, shuffle=False)

In [ ]:
for inputs, targets in trainloader:
    inputs, targets = inputs.cuda(), targets.cuda()
    break
x = inputs.permute(0, 2, 3, 1)
print(x.shape)
print(targets.shape)

In [ ]:
x.mean(dim=(0, 1, 2))
#p = net(inputs)['out']
#p.max()

In [ ]:
x.std(dim=(0, 1, 2))

In [ ]:
x.mean(dim=(1, 2, 3))

In [ ]:
import pytorch_unet

In [ ]:
unet = pytorch_unet.UNet(1).cuda()

In [ ]:
with torch.no_grad():
    net(inputs)

In [ ]:
'''
relu = nn.ReLU()
image_mean = x.mean(dim=(1, 2, 3), keepdim=True)
variance = x**2 - image_mean**2
variance = relu(variance)  # this relu is critical for grad wrt x
stddev = torch.sqrt(variance)
min_stddev = torch.rsqrt(
    torch.prod(torch.FloatTensor([x.size()[1:]]))).cuda()
pixel_value_scale = torch.max(stddev, min_stddev)
x = x - image_mean
#x = x / pixel_value_scale
'''

In [ ]:
x.max()

In [ ]:
idx = 1
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

img = x.detach().cpu().numpy()[idx]

ax1.imshow(img)
#ax2.imshow(targets.detach().cpu().numpy()[idx])
#ax2.imshow(variance[idx].detach().cpu().numpy())

In [ ]:
x.mean(dim=(1, 2, 3))

In [ ]:
bs = 10
valloader = torch.utils.data.DataLoader(valset, batch_size=bs, shuffle=False)

In [ ]:
val_targets = torch.zeros((targets.size(0), targets.size(1), targets.size(2)))
print(val_targets.size())

In [ ]:
#val_targets.min()

In [ ]:
val_iou = 0
val_loss = 0
for inputs, targets in valloader:
    inputs, targets = inputs.cuda(), targets.cuda()
    val_targets = torch.zeros(
        (targets.size(0), targets.size(1), targets.size(2)), 
        dtype=torch.long).cuda()
    val_targets[targets[:, :, :, 0] == 128] = 1
    with torch.no_grad():
        pred = net(inputs)['out'] # fprop
        batch_iou = eval_binary_iou(sig(pred), val_targets)
        val_iou += batch_iou.item()
        # dataloader outputs targets with shape NHW, but we need NCHW
        val_targets = val_targets.unsqueeze(dim=1).float()
        batch_loss = loss_fn(pred, val_targets)
        val_loss += batch_loss.item()
        print(batch_iou)
val_loss /= len(valloader)
val_iou /= len(valloader)
print('Loss: {:.4f}, IoU: {:.4f}'.format(val_loss, val_iou))

In [ ]:
#loss_fn(pred, targets.unsqueeze(1).float())

In [ ]:
#inputs.max()

In [ ]:
#np.unique(targets.detach().cpu().numpy()[1, 0])

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib inline

In [ ]:
#plt.hist(pred.detach().cpu().numpy().reshape(-1))

In [ ]:
outputs = pred.squeeze(1).round().long()
#outputs.shape

In [ ]:
#figsize=(9, 3)
#figsize=(6, 3)
#idx = 2 # train mode
for idx in range(bs):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4)
    ax1.imshow(inputs_nhwc.detach().cpu().numpy()[idx])
    ax2.imshow(pred.detach().cpu().numpy()[idx, 0])
    ax3.imshow(pred.round().detach().cpu().numpy()[idx, 0])
    #ax3.imshow(union.detach().cpu().numpy()[idx])
    ax4.imshow(targets.detach().cpu().numpy()[idx])
    
    #lbl = targets[idx].cpu().numpy().reshape(1, -1)
    #out = outputs[idx].cpu().numpy().reshape(1, -1)
    #jscv = jsc(out, lbl)

    #ax1.set_title('Image'); ax2.set_title('Logits') 
    #ax3.set_title('IoU %.2f' % iou[idx]); 
    #ax4.set_title('IoU %.2f' % jscv)
    pretty_image([ax1, ax2, ax3, ax4])
    #plt.savefig(resume.split('/')[-1].split('.')[0] + '_demo_preds_train' + str(net.training) + '_%d.png' % idx)

# Workspace

In [ ]:
#outputs.round().int()

In [ ]:
# zero if output=0 or pred=0
outputs = pred.squeeze(1).round().long()
eps = 1e-6
intersection = (outputs & targets).float().sum((1, 2))
print(intersection)
union = (outputs | targets).float().sum((1, 2))
print(union)
iou = intersection / (union + eps)
print(iou.mean())

In [ ]:
#intersection = torch.m(outputs == targets).float().sum((1, 2))
#print(intersection)

In [ ]:
print(iou.mean())

In [ ]:
# version 0.20.2
from sklearn.metrics import jaccard_similarity_score as jsc

In [ ]:
jsc(np.array([1, 1]), np.array([1, 1]))
jsc(np.array([1, 1]), np.array([1, 0]))
jsc(np.array([1, 1, 0]), np.array([1, 0, 0]))
jsc(np.array([[1, 1, 0]]), np.array([[1, 0, 0]]))

In [ ]:
idx = 0
lbl = targets[0].cpu().numpy().reshape(1, -1)
out = outputs[0].cpu().numpy().reshape(1, -1)
jscv = jsc(out, lbl)
print(jscv)

In [ ]:
lbl.shape

In [ ]:
targets.shape

In [ ]:
soft_intersection = torch.mul(pred.squeeze(1), targets.squeeze(1)).sum((1, 2))
print(soft_intersection)

In [ ]:
soft_union = (pred.squeeze(1) + targets.squeeze(1)).sum((1, 2)) - soft_intersection
soft_union

In [ ]:
(soft_intersection / soft_union).mean()

In [ ]:
iou_np = iou.detach().cpu().numpy()

In [ ]:
#inputs_nhwc = inputs.permute(0, 2, 3, 1)

In [ ]:
def pretty_image(axes):
    for ax in axes:
        ax.axis('off')
    plt.tight_layout()

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, weight_decay=5e-4)

In [ ]:
for param_group in optimizer.param_groups:
    print(param_group)

In [ ]:
param_group['lr'] = 1e-3

In [ ]:
param_group['lr']

In [ ]:
epochs = 1
for epoch in range(epochs):
    for batch, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        pred = net(inputs)['out']
        
        loss = loss_fn(pred, 
                       torch.LongTensor(
                           targets.unsqueeze(1) * 255)
                      )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'
          .format(epoch + 1, epochs, loss.item()))

In [ ]:
#pixel_acc(targets, targets)
pixel_acc(pred, pred)

In [ ]:
targets.shape

In [ ]:
def pixel_acc(pred, label):
    #_, preds = torch.max(pred, dim=1)
    preds = torch.argmax(pred, dim=1)
    valid = (label >= 0).long()
    acc_sum = torch.sum(valid * (preds == label).long())
    pixel_sum = torch.sum(valid)
    acc = acc_sum.float() / (pixel_sum.float() + 1e-10)
    return acc

In [ ]:
class SegmentationModuleBase(nn.Module):
    def __init__(self):
        super(SegmentationModuleBase, self).__init__()

    def pixel_acc(self, pred, label):
        _, preds = torch.max(pred, dim=1)
        valid = (label >= 0).long()
        acc_sum = torch.sum(valid * (preds == label).long())
        pixel_sum = torch.sum(valid)
        acc = acc_sum.float() / (pixel_sum.float() + 1e-10)
        return acc


class SegmentationModule(SegmentationModuleBase):
    def __init__(self, net_enc, net_dec, crit, deep_sup_scale=None):
        super(SegmentationModule, self).__init__()
        self.encoder = net_enc
        self.decoder = net_dec
        self.crit = crit
        self.deep_sup_scale = deep_sup_scale

    def forward(self, feed_dict, *, segSize=None):
        # training
        if segSize is None:
            if self.deep_sup_scale is not None: # use deep supervision technique
                (pred, pred_deepsup) = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True))
            else:
                pred = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True))

            loss = self.crit(pred, feed_dict['seg_label'])
            if self.deep_sup_scale is not None:
                loss_deepsup = self.crit(pred_deepsup, feed_dict['seg_label'])
                loss = loss + loss_deepsup * self.deep_sup_scale

            acc = self.pixel_acc(pred, feed_dict['seg_label'])
            return loss, acc
        # inference
        else:
            pred = self.decoder(self.encoder(feed_dict['img_data'], return_feature_maps=True), segSize=segSize)
            return pred